In [17]:
import numpy as np
import math
import pyfirmata
import time

In [18]:
# initialisation

l1 = 142; l2= 265; l3 = 210; l4 = 130;
xg = 200
yg = -200
zg = 200

# orientation wrt base frame
yaw = 45*(np.pi/180);    # rotation about Z axis
pitch = 180*(np.pi/180);   # rotation about Y axis
roll = 0*(np.pi/180);    # rotation about X axis

# initial rotation matrix

Rx = np.array([[1,0,0],
      [0, math.cos(roll), -math.sin(roll)],
      [0, math.sin(roll), math.cos(roll)]])

Ry = np.array([[math.cos(pitch),0,math.sin(pitch)],
      [0, 1, 0],
      [-math.sin(pitch), 0, math.cos(pitch)]])

Rz = np.array([[math.cos(yaw),-math.sin(yaw),0],
      [math.sin(yaw), math.cos(yaw), 0],
      [0, 0, 1]])
R60 = np.dot(Rx,Ry,Rz)

print(R60)

# wrist and goal Positions
Pg = np.array([[xg],[yg],[zg]])
print(Pg)                              # goal position vector
Pw = Pg -(l4*R60[:,2].reshape(3,1))    # wrist position vector
print(Pw)                             

# initial transformation matrix T60
T60_temp = np.hstack((R60,Pw))
last_row = np.array([0,0,0,1])
T60 = np.vstack((T60_temp,last_row))
print(T60)              


[[-1.0000000e+00  0.0000000e+00  1.2246468e-16]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [-1.2246468e-16  0.0000000e+00 -1.0000000e+00]]
[[ 200]
 [-200]
 [ 200]]
[[ 200.]
 [-200.]
 [ 330.]]
[[-1.0000000e+00  0.0000000e+00  1.2246468e-16  2.0000000e+02]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00 -2.0000000e+02]
 [-1.2246468e-16  0.0000000e+00 -1.0000000e+00  3.3000000e+02]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


In [19]:
# Inverse Kinematics........... Inverse Positioning

theta1 = math.atan2(Pw[1],Pw[0]) # angle for joint1

C3 = (Pw[0]**2 + Pw[1]**2 + (Pw[2]-l1)**2 -l2**2 -l3**2)/(2*l2*l3) # cosine of theta3
S3 = math.sqrt(1-C3**2)                                            # sine of theta3
theta3 = math.atan2(S3,C3)

# From squaring x,y,z, terms and solving for theta2
A  = Pw[0]**2 + Pw[1]**2 + Pw[2]**2 - l1**2 -l2**2 - l3**2 -2*l2*l3*C3
alpha = 2*l1*l3*C3
beta = 2*l1*l3*S3
gamma = 2*l1*l2
a2 = (alpha+gamma)**2 + beta**2
b2 = -2*A*(alpha+gamma)
c2 = A**2 - beta**2

C2 =(-b2+ math.sqrt(b2**2-4*(a2*c2)))/(2*a2) # cosine of theta 2 roots([a2 b2 c2])
S2 = math.sqrt(1-C2[0]**2)  # Sine of theta2
theta2 = math.atan2(S2,C2);

print(theta1*180/np.pi)
print(theta2*180/np.pi)
print(theta3*180/np.pi)

-45.00000000000001
18.19624552227852
89.47542484272996


In [20]:
# determining Transformation Matrices T30 and T63

T30 = np.array([[math.cos(theta1)*math.cos(theta2 - np.pi/2)*math.cos(theta3 + np.pi/2)-math.cos(theta1)*math.sin(theta2 - np.pi/2)*math.sin(theta3 + np.pi/2), -math.cos(theta1)*math.cos(theta2 - np.pi/2)*math.sin(theta3 + np.pi/2) - math.cos(theta1)*math.cos(theta3 + np.pi/2)*math.sin(theta2 - np.pi/2), -math.sin(theta1), l2*math.cos(theta1)*math.cos(theta2 - np.pi/2)],
[math.cos(theta2 - np.pi/2)*math.cos(theta3 + np.pi/2)*math.sin(theta1) - math.sin(theta1)*math.sin(theta2 - np.pi/2)*math.sin(theta3 + np.pi/2), - math.cos(theta2 - np.pi/2)*math.sin(theta1)*math.sin(theta3 + np.pi/2) - math.cos(theta3 +np.pi/2)*math.sin(theta1)*math.sin(theta2 - np.pi/2),  math.cos(theta1), l2*math.cos(theta2 - np.pi/2)*math.sin(theta1)],
[              - math.cos(theta2 - np.pi/2)*math.sin(theta3 + np.pi/2) - math.cos(theta3 + np.pi/2)*math.sin(theta2 - np.pi/2),                   math.sin(theta2 - np.pi/2)*math.sin(theta3 + np.pi/2) - math.cos(theta2 - np.pi/2)*math.cos(theta3 + np.pi/2),        0,  l1 - l2*math.sin(theta2 - np.pi/2)],
[                                                                            0,                                                                               0,        0,                           1]
 ])

R30 = T30[0:3,0:3]

# Calculate R63 matrix using R30 inverse and R60
R63 = R30.T @ R60

print('R30 = ',R30)
print('R63 = ',R63)

R30 =  [[-0.21465074 -0.67373961  0.70710678]
 [ 0.21465074  0.67373961  0.70710678]
 [-0.95281169  0.30356198  0.        ]]
R63 =  [[ 2.14650738e-01  2.14650738e-01  9.52811693e-01]
 [ 6.73739609e-01  6.73739609e-01 -3.03561985e-01]
 [-7.07106781e-01  7.07106781e-01  8.65956056e-17]]


In [21]:
# Inverse Orientation

#calculation for theta5
C5 = -R63[1, 2] # Cosine of theta5
S5 = math.sqrt(R63[2, 2]**2 + R63[0, 2]**2) # Sine of theta5
theta5 = math.atan2(S5, C5)

# computing theta 6 by comparing values in R63
S6 = -R63[1,1]/S5
C6 = R63[1,0]/S5
theta6 = math.atan2(S6,C6)

# computing theta4 by comparing values in R63
S4 = R63[2,2]/S5
C4 = R63[0,2]/S5
theta4 = math.atan2(S4,C4)


In [22]:
print('Theta1 = ',theta1*(180/np.pi))
print('Theta2 = ',theta2*(180/np.pi))
print('Theta3 = ',theta3*(180/np.pi))
print('Theta4 = ',theta4*(180/np.pi))
print('Theta5 = ',theta5*(180/np.pi))
print('Theta6 = ',theta6*(180/np.pi))

Theta1 =  -45.00000000000001
Theta2 =  18.19624552227852
Theta3 =  89.47542484272996
Theta4 =  5.207285726468571e-15
Theta5 =  72.32832963499152
Theta6 =  -45.00000000000001


In [23]:
# adjusting gear ratios and angles

theta1 = (theta1) *40
theta2 = theta2 *44
theta3 = -(theta3-90) *40
theta4 = (-theta4) *308
theta5 = theta5 *12
theta6 = -(theta6) *28

In [ ]:
# arduino motor control
# Change to the appropriate serial port
port = 'COM9'

# calculate steps for each angle
step1 = int(theta1*200/360);
step2 = int(theta2*200/360);
step3 = int(theta3*200/360);
step4 = int(theta4*200/360);
step5 = int(theta5*200/360);
step6 = int(theta6*200/360);

# Create a new board instance
board = pyfirmata.Arduino(port)

# Start an iterator thread to prevent buffer overflow
it = pyfirmata.util.Iterator(board)
it.start()

# Define stepper motor pins
#Adjust these pins according to your motor driver connections
# step_pin1 = board.get_pin('d:49:o')  # Digital pin for step
# dir_pin1 = board.get_pin('d:48:o')   # Digital pin for direction
step_pin2 = board.get_pin('d:10:o')  # Digital pin for step
dir_pin2 = board.get_pin('d:11:o')    # Digital pin for direction
# step_pin3 = board.get_pin('d:52:o')  # Digital pin for step
# dir_pin3 = board.get_pin('d:53:o')   # Digital pin for direction
# step_pin4 = board.get_pin('d:34:o')  # Digital pin for step
# dir_pin4 = board.get_pin('d:35:o')   # Digital pin for direction
# step_pin5 = board.get_pin('d:38:o')  # Digital pin for step
# dir_pin5 = board.get_pin('d:39:o')   # Digital pin for direction
# step_pin6 = board.get_pin('d:42:o')  # Digital pin for step
# dir_pin6 = board.get_pin('d:43:o')   # Digital pin for direction

def stepper_step(steps, direction,step_pin,dir_pin, step_delay=0.01):
    dir_pin.write(direction)
    for _ in range(steps):
        step_pin.write(1)
        time.sleep(step_delay)
        step_pin.write(0)
        time.sleep(step_delay)

# Example usage: move the stepper motor
try:
    while True:
        # initialize
#         stepper_step(2000, 1,step_pin3,dir_pin3,0.01)
#         time.sleep(1)
#         stepper_step(-200, 1,step_pin5,dir_pin5,0.01)
#         time.sleep(1)
        
        # Move stepper
#         stepper_step(step1, 1,step_pin1,dir_pin1,0.01)
#         time.sleep(1)
        stepper_step(step2, 1,step_pin2,dir_pin2,0.01)
        time.sleep(1)
#         stepper_step(step3, 1,step_pin3,dir_pin3,0.01)
#         time.sleep(1)
#         stepper_step(step4, 1,step_pin4,dir_pin4,0.01)
#         time.sleep(1)
#         stepper_step(step5, 1,step_pin5,dir_pin5,0.01)
#         time.sleep(1)
#         stepper_step(step6, 1,step_pin6,dir_pin6,0.01)
#         time.sleep(1)
        
        # Retract
#         stepper_step(-step1, 1,step_pin1,dir_pin1,0.01)
#         time.sleep(1)
#         stepper_step(-step2, 1,step_pin2,dir_pin2,0.01)
#         time.sleep(1)
#         stepper_step(-step3, 1,step_pin3,dir_pin3,0.01)
#         time.sleep(1)
#         stepper_step(-step4, 1,step_pin4,dir_pin4,0.01)
#         time.sleep(1)
#         stepper_step(-step5, 1,step_pin5,dir_pin5,0.01)
#         time.sleep(1)
#         stepper_step(-step6, 1,step_pin6,dir_pin6,0.01)
#         time.sleep(1)

        # homing
#         stepper_step(-2000, 1,step_pin3,dir_pin3,0.01)
#         time.sleep(1)
#         stepper_step(200, 1,step_pin5,dir_pin5,0.01)
#         time.sleep(1)

except KeyboardInterrupt:
    print("Exiting...")

finally:
    board.exit()